# Lab-11-2 RNN hihello and charseq

### 'hihello' problem

'h', 'i', 'h', 'e', 'l', 'l', 'o'

We will predict the next character!

How can we represent characters in pyTorch?

We can represent them by index

'h' -> 0<br>
'i' -> 1<br>
'e' -> 2<br>
'l' -> 3<br>
'o' -> 4

In [2]:
# list of avialable characters
char_set = ['h', 'i', 'e', 'l', 'o']

But, continuous 하지 않고 categotical한 데이터를 표현하고자 할 때는 주로 one-hot encoing 사용

In [ ]:
# list of avialable characters
char_set = ['h', 'i', 'e', 'l', 'o']
x_data = [[0, 1, 0, 2, 3, 3]]
x_one_hot = [[[1, 0, 0, 0, 0],
             [0, 1, 0, 0, 0],
             [1, 0, 0, 0, 0],
             [0, 0, 1, 0, 0],
             [0, 0, 0, 1, 0],
             [0, 0, 0, 1, 0]]]
y_data =  [[1, 0, 2, 3, 3, 4]]

# x_data는 입력이므로 마지막 'o' 표시 x
# y_data는 출력이므로 처음 'h' 표시 x

### Cross Entropy Loss

loss for categorical output (usuallt interpreted as probability)

model -> output :[[0.1], [0.2], [0.3], [0.4]] label: [0, 0, 0, 1]

# loss & optimizer setting
criterion = torch.nn.CrossEntropyLoss()

...

loss = criterion(outputs.view(-1, input_size), Y.view(-1))

In [3]:
# 첫 번째 파라미터: 모델의 아웃풋
# 두 번째 파라미터: 정답 레이블

### Code run through (hihello)

In [5]:
char_set = ['h', 'i', 'e', 'l', 'o']

# hyper parameters
input_size = len(char_set)
hidden_size = len(char_set)
learing_rate = 0.1

# data setting
x_data = [[0, 1, 0, 2, 3, 3]]
x_one_hot = [[[1, 0, 0, 0, 0],
             [0, 1, 0, 0, 0],
             [1, 0, 0, 0, 0],
             [0, 0, 1, 0, 0],
             [0, 0, 0, 1, 0],
             [0, 0, 0, 1, 0]]]
y_data =  [[1, 0, 2, 3, 3, 4]]

# transform as torch tensor variable
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

### Code run through (charseq)

In [9]:
sample = 'if you want you'
# make dictionary
char_set = list(set(sample))
char_dic = {c:  i for i,c in enumerate(char_set)}

# hyper parameters
dic_size = len(char_dic)
hidden_size = len(char_dic)
learning_rate = 0.1

# data setting
sample_idx = [char_dic[c] for c in sample]
x_data = [sample_idx[:-1]]
x_one_hot = [np.eye(dic_size)[x] for x in x_data]
y_data = [sample_idx[1:]]

# transform as torch tensor variable
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [12]:
char_set

['t', 'a', 'i', ' ', 'f', 'y', 'u', 'w', 'n', 'o']

### code run through

In [15]:
# declare RNN
rnn = torch.nn.RNN(dic_size, hidden_size, batch_first = True) # batch_first gurantees the order of output = (B, S, F)

# loss & optimizer setting
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), learning_rate)

#start training
for i in range(100):
    optimizer.zero_grad() # RNN에서는 처음에 그레디언트 벡터 초기화 필수
    outputs, _status = rnn(X)
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()
    result = outputs.data.numpy().argmax(axis = 2)
    result_str = ''.join([char_set[c] for c in np.squeeze(result)])
    print(i, 'loss: ', loss.item(), 'prediction: ', result, 'true Y: ', y_data, "prediction str: ", result_str)

0 loss:  2.444728374481201 prediction:  [[2 2 2 2 7 2 4 0 0 0 7 0 2 7]] true Y:  [[4, 3, 5, 9, 6, 3, 7, 1, 8, 0, 3, 5, 9, 6]] prediction str:  iiiiwiftttwtiw
1 loss:  2.0261826515197754 prediction:  [[5 2 5 3 5 3 5 4 2 5 3 5 3 5]] true Y:  [[4, 3, 5, 9, 6, 3, 7, 1, 8, 0, 3, 5, 9, 6]] prediction str:  yiy y yfiy y y
2 loss:  1.7061254978179932 prediction:  [[5 3 5 3 5 3 7 9 7 5 3 5 3 6]] true Y:  [[4, 3, 5, 9, 6, 3, 7, 1, 8, 0, 3, 5, 9, 6]] prediction str:  y y y wowy y u
3 loss:  1.5431398153305054 prediction:  [[5 3 5 9 6 3 7 9 8 5 3 5 9 6]] true Y:  [[4, 3, 5, 9, 6, 3, 7, 1, 8, 0, 3, 5, 9, 6]] prediction str:  y you wony you
4 loss:  1.452165961265564 prediction:  [[5 3 5 9 6 3 7 9 8 5 3 6 9 6]] true Y:  [[4, 3, 5, 9, 6, 3, 7, 1, 8, 0, 3, 5, 9, 6]] prediction str:  y you wony uou
5 loss:  1.379278540611267 prediction:  [[4 3 5 9 6 3 7 9 8 0 3 6 3 6]] true Y:  [[4, 3, 5, 9, 6, 3, 7, 1, 8, 0, 3, 5, 9, 6]] prediction str:  f you wont u u
6 loss:  1.3213716745376587 prediction:  [[4 3 5 

In [16]:
outputs.data.numpy()

array([[[-0.99496174, -0.989334  , -0.99630564, -0.980675  ,
          0.9988039 , -0.9963068 , -0.9929592 , -0.99827224,
         -0.99652845, -0.9971067 ],
        [-0.9998039 , -0.96539265, -0.9999935 ,  0.99915266,
         -0.9998175 , -0.9998145 , -0.99988073, -0.9998098 ,
         -0.9963792 , -0.99929017],
        [-0.9996685 , -0.99972445, -0.99999994, -0.99958676,
         -0.9999952 ,  0.9903841 , -0.9991908 , -0.925429  ,
         -0.9999996 , -0.999994  ],
        [-0.9999804 , -0.9996409 , -0.9999983 , -0.99652386,
         -0.99999803, -0.9997043 , -0.99801666, -0.9999978 ,
         -0.9999955 ,  0.9993632 ],
        [-0.9998848 , -0.999968  , -0.9999999 , -0.9985639 ,
         -0.9999888 , -0.99999994,  0.999241  , -0.9995045 ,
         -0.9969242 , -0.9992147 ],
        [-0.9977456 , -0.9991373 , -0.9999999 ,  0.999228  ,
         -0.9961742 , -1.        , -0.10205668, -0.98498034,
         -0.99999183, -0.40980852],
        [-0.99935615, -0.99925464, -0.9999999 , -0.9

In [22]:
result = outputs.data.numpy().argmax(axis = 2)
result
np.squeeze(result)

array([4, 3, 5, 9, 6, 3, 7, 1, 8, 0, 3, 5, 9, 6], dtype=int64)